# Fully Connected Neural Networks

## Introduction

Fully Connected Neural Networks are the most basic form of Neural Networks. They are also known as Multi-layered Perceptrons (MLP). They are called fully connected because each neuron in one layer is connected to every neuron in the next layer. The input layer is the first layer and the output layer is the last layer. The layers in between are called hidden layers. The number of hidden layers and the number of neurons in each layer are hyperparameters that need to be tuned.

全连接神经网络是最基本的神经网络形式。它们也被称为多层感知器（MLP）。它们被称为全连接，因为每一层中的每个神经元都与下一层中的每个神经元相连。输入层是第一层，输出层是最后一层。中间的层称为隐藏层。隐藏层的数量和每层的神经元数量是需要调整的超参数。

Usually, MLP is used for classification problems. The input is a feature vector and the output is a class label. The output layer has as many neurons as the number of classes. The output of each neuron in the output layer is the probability of the input belonging to that class. The class with the highest probability is the predicted class.

通常，MLP用于分类问题。输入是特征向量，输出是类标签。输出层的神经元数量与类的数量相同。输出层中每个神经元的输出是输入属于该类的概率。具有最高概率的类是预测类。

In [ ]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import pytorch
import torch
import torch.nn as nn